# Business Problem:
Hospitals and diagnostic centers generate vast amounts of unstructured medical text data, including clinical notes, pathology reports, and radiology findings. Manually reviewing these documents to detect brain tumors is a time-consuming and error-prone task that requires specialized medical expertise. As the volume of patient data increases, healthcare professionals face growing challenges in efficiently processing and analyzing this information.

This project aims to address the manual bottleneck by automating the classification of medical text reports related to brain tumors using Convolutional Neural Networks (CNNs). By leveraging deep learning techniques, the model can automatically detect the presence and type of brain tumors, significantly enhancing diagnostic accuracy and speed. This solution empowers healthcare providers to:

**Reduce human error** in identifying brain tumor cases from complex medical reports.

**Streamline diagnosis workflows**, allowing clinicians and radiologists to focus on treatment rather than document categorization.

**Accelerate treatment planning** by providing faster and more reliable tumor detection, improving overall patient outcomes and operational efficiency.



In [1]:
import tensorflow as tf
import keras

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_datagen = ImageDataGenerator(rescale = 1/255,
                                  shear_range =0.2,
                                  zoom_range = 0.2)

In [4]:
training_set = train_datagen.flow_from_directory(r'D:\Brain Tumer Image\dataset\train',
                                                target_size = (64, 64),
                                                 color_mode='grayscale',  # ← this is the key change
                                                 class_mode='binary')

Found 5542 images belonging to 2 classes.


In [5]:
training_set.class_indices

{'no_tumor': 0, 'tumor': 1}

In [6]:
test_datagen = ImageDataGenerator(rescale =1/255)

test_set = test_datagen.flow_from_directory(r'D:\Brain Tumer Image\dataset\validation',
                                           target_size = (64,64),
                                            color_mode='grayscale',  # ← match here too
                                            class_mode = 'binary')

Found 1404 images belonging to 2 classes.


# Modelling - Convolution Neural Network

**Initialising the CNN**

In [7]:
from keras.models import Sequential
classifier = Sequential()

**Step 1 - Convolution**

In [8]:
from keras.layers import Conv2D
classifier.add(Conv2D(input_shape=[64, 64, 1],
                      filters = 32, kernel_size =3, activation ='relu'))

C:\Users\NAGARJUNA\anaconda3\nagaraju\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Step 2 - MaxPooling**

In [9]:
from keras.layers import MaxPooling2D
classifier.add(MaxPooling2D(pool_size=2, strides=2))

**Step 3 - Flattening**

In [10]:
from keras.layers import Flatten
classifier.add(Flatten())

**Step 4 - Full Connection**

In [11]:
from keras.layers import Dense

# hidden layers with 128 neurons
classifier.add(Dense(units= 128, activation='relu'))

# Output Layer with 1 Neuron
classifier.add(Dense(units =1, activation ='sigmoid'))

**Training the CNN Model with train data & testing the Model with test data**

In [12]:
classifier.compile(optimizer='adam',
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])

In [13]:
classifier.fit(x=training_set, validation_data = test_set, epochs = 25)

C:\Users\NAGARJUNA\anaconda3\nagaraju\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
174/174 ━━━━━━━━━━━━━━━━━━━━ 444s 3s/step - accuracy: 0.7567 - loss: 0.4993 - val_accuracy: 0.9110 - val_loss: 0.2292
Epoch 2/25
174/174 ━━━━━━━━━━━━━━━━━━━━ 41s 234ms/step - accuracy: 0.9377 - loss: 0.1726 - val_accuracy: 0.8803 - val_loss: 0.2965
Epoch 3/25
174/174 ━━━━━━━━━━━━━━━━━━━━ 40s 229ms/step - accuracy: 0.9505 - loss: 0.1469 - val_accuracy: 0.9601 - val_loss: 0.1213
Epoch 4/25
174/174 ━━━━━━━━━━━━━━━━━━━━ 40s 231ms/step - accuracy: 0.9562 - loss: 0.1209 - val_accuracy: 0.9601 - val_loss: 0.1216
Epoch 5/25
174/174 ━━━━━━━━━━━━━━━━━━━━ 40s 231ms/step - accuracy: 0.9671 - loss: 0.1034 - val_accuracy: 0.9430 - val_loss: 0.1497
Epoch 6/25
174/174 ━━━━━━━━━━━━━━━━━━━━ 40s 232ms/step - accuracy: 0.9708 - loss: 0.0834 - val_accuracy: 0.9679 - val_loss: 0.1196
Epoch 7/25
174/174 ━━━━━━━━━━━━━━━━━━━━ 40s 231ms/step - accuracy: 0.9692 - loss: 0.0868 - val_accuracy: 0.9494 - val_loss: 0.1439
Epoch 8/25
174/174 ━━━━━━━━━━━━━━━━━━━━ 41s 233ms/step - accuracy: 0.9766 - loss: 0.0

# Evaluation 

- **Making a Single prediction**

In [14]:
from PIL import Image
import numpy as np

# Load and preprocess the image
test_image = Image.open(r"D:\Brain Tumer Image\Image (138).jpg")
test_image = test_image.convert('L')  # convert to grayscale
test_image = test_image.resize((64, 64))

test_image = np.array(test_image)
test_image = test_image.reshape(64, 64, 1)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image.astype('float32') / 255.0  # Normalize the image

# Prediction
result = classifier.predict(test_image)

# Evaluation
if result[0][0] == 1:
    print("Tumor Detected")
else:
    print("No Tumor Detected")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
No Tumor Detected
